## Student Information

**Name:** [Your Name Here]  
**Roll Number:** [Your Roll Number]  
**Date:** [Submission Date]  
**College:** [Your College Name]  
**Academic Year:** 2025-2026  

---

## Learning Outcomes Checklist

- [ ] LO1: Split data into train and test sets appropriately
- [ ] LO2: Implement Decision Tree Classifier
- [ ] LO3: Implement K-Nearest Neighbors (KNN)
- [ ] LO4: Implement Logistic Regression
- [ ] LO5: Evaluate using confusion matrix and classification reports
- [ ] LO6: Calculate precision, recall, F1-score metrics
- [ ] LO7: Compare multiple classifiers
- [ ] LO8: Tune hyperparameters
- [ ] LO9: Visualize model performance
- [ ] LO10: Select best model based on metrics

## Phase 0: Environment Setup & Library Imports

In [ ]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (confusion_matrix, classification_report, 
                             accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, roc_curve, auc)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Print versions
print("📚 LIBRARY VERSIONS:")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Scikit-learn: {__import__('sklearn').__version__}")
print("\n✅ All libraries imported successfully!")

# Configure display
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## Phase 1: Data Preparation & Exploration

Prepare a classification dataset for model training.

In [ ]:
# Create a classification dataset
from sklearn.datasets import make_classification, load_iris

print("📊 CREATING CLASSIFICATION DATASET:\n")

# Use Iris dataset for classification
X, y = load_iris(return_X_y=True)
feature_names = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']
target_names = ['Setosa', 'Versicolor', 'Virginica']

# Create DataFrame
df = pd.DataFrame(X, columns=feature_names)
df['Species'] = y
df['Species_Name'] = df['Species'].map({i: target_names[i] for i in range(3)})

print(f"Dataset shape: {df.shape}")
print(f"\nFeatures: {feature_names}")
print(f"Target classes: {target_names}")
print(f"\nClass distribution:")
print(df['Species_Name'].value_counts())
print(f"\nFirst 5 rows:")
print(df.head())

In [ ]:
# Data exploration
print("📈 DATA EXPLORATION:\n")

print(f"Statistics:")
print(df.describe())

print(f"\nMissing values: {df.isnull().sum().sum()}")
print(f"Data types:\n{df.dtypes}")

## Phase 2: Train-Test Split

Split data into training and testing sets.

In [ ]:
# Prepare features and target
print("🔀 TRAIN-TEST SPLIT:\n")

X = df[feature_names]
y = df['Species']

# Split data: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")
print(f"\nTraining set distribution:")
print(pd.Series(y_train).value_counts().sort_index())
print(f"\nTesting set distribution:")
print(pd.Series(y_test).value_counts().sort_index())
print(f"\n✅ Data split successfully!")

In [ ]:
# Feature scaling (important for KNN and Logistic Regression)
print("\n📏 FEATURE SCALING:\n")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training data - Mean: {X_train_scaled.mean(axis=0):.2f}")
print(f"Training data - Std: {X_train_scaled.std(axis=0):.2f}")
print(f"\n✅ Features scaled successfully!")

## Phase 3: Decision Tree Classifier

Implement and evaluate Decision Tree for classification.

In [ ]:
# Train Decision Tree
print("🌳 DECISION TREE CLASSIFIER:\n")

dt_model = DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

dt_model.fit(X_train, y_train)

# Make predictions
y_pred_dt = dt_model.predict(X_test)

# Calculate accuracy
dt_accuracy = accuracy_score(y_test, y_pred_dt)
print(f"✅ Decision Tree trained!")
print(f"Accuracy: {dt_accuracy:.4f} ({dt_accuracy*100:.2f}%)")
print(f"\nHyperparameters:")
print(f"  - Max depth: 5")
print(f"  - Min samples split: 2")
print(f"  - Min samples leaf: 1")

In [ ]:
# Feature importance from Decision Tree
print("\n📊 FEATURE IMPORTANCE:\n")

feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': dt_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(feature_importance.to_string(index=False))

# Visualize
plt.figure(figsize=(10, 5))
plt.bar(feature_importance['Feature'], feature_importance['Importance'])
plt.title('Feature Importance - Decision Tree', fontsize=14, fontweight='bold')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n✅ Feature importance calculated!")

## Phase 4: K-Nearest Neighbors (KNN)

Implement and evaluate KNN classifier.

In [ ]:
# Train KNN model
print("👥 K-NEAREST NEIGHBORS (KNN):\n")

knn_model = KNeighborsClassifier(
    n_neighbors=5,
    weights='uniform',
    metric='euclidean'
)

# Train on scaled data
knn_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_knn = knn_model.predict(X_test_scaled)

# Calculate accuracy
knn_accuracy = accuracy_score(y_test, y_pred_knn)
print(f"✅ KNN trained!")
print(f"Accuracy: {knn_accuracy:.4f} ({knn_accuracy*100:.2f}%)")
print(f"\nHyperparameters:")
print(f"  - K neighbors: 5")
print(f"  - Weights: uniform")
print(f"  - Metric: euclidean")

In [ ]:
# Find optimal K
print("\n🔍 FINDING OPTIMAL K:\n")

k_values = range(1, 16)
train_scores = []
test_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_scores.append(knn.score(X_train_scaled, y_train))
    test_scores.append(knn.score(X_test_scaled, y_test))

# Plot K vs Accuracy
plt.figure(figsize=(10, 5))
plt.plot(k_values, train_scores, marker='o', label='Train Accuracy')
plt.plot(k_values, test_scores, marker='s', label='Test Accuracy')
plt.xlabel('K Value')
plt.ylabel('Accuracy')
plt.title('KNN: Effect of K on Accuracy', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

best_k = k_values[np.argmax(test_scores)]
print(f"Optimal K: {best_k}")
print(f"Best test accuracy: {max(test_scores):.4f}")

## Phase 5: Logistic Regression

Implement and evaluate Logistic Regression classifier.

In [ ]:
# Train Logistic Regression
print("📈 LOGISTIC REGRESSION:\n")

lr_model = LogisticRegression(
    max_iter=200,
    multi_class='multinomial',
    random_state=42
)

# Train on scaled data
lr_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_lr = lr_model.predict(X_test_scaled)

# Calculate accuracy
lr_accuracy = accuracy_score(y_test, y_pred_lr)
print(f"✅ Logistic Regression trained!")
print(f"Accuracy: {lr_accuracy:.4f} ({lr_accuracy*100:.2f}%)")
print(f"\nHyperparameters:")
print(f"  - Max iterations: 200")
print(f"  - Multi-class: multinomial")
print(f"  - Solver: lbfgs (default)")

In [ ]:
# Coefficient analysis
print("\n📊 COEFFICIENT ANALYSIS:\n")

coefficients = pd.DataFrame(
    lr_model.coef_,
    columns=feature_names,
    index=target_names
)

print("Logistic Regression Coefficients:")
print(coefficients)
print(f"\nIntercept (bias): {lr_model.intercept_}")

## Phase 6: Model Evaluation

Comprehensive evaluation of all three models.

In [ ]:
# Confusion matrices
print("🎯 CONFUSION MATRICES:\n")

cm_dt = confusion_matrix(y_test, y_pred_dt)
cm_knn = confusion_matrix(y_test, y_pred_knn)
cm_lr = confusion_matrix(y_test, y_pred_lr)

print("Decision Tree Confusion Matrix:")
print(cm_dt)
print("\nKNN Confusion Matrix:")
print(cm_knn)
print("\nLogistic Regression Confusion Matrix:")
print(cm_lr)

In [ ]:
# Visualize confusion matrices
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

matrices = [cm_dt, cm_knn, cm_lr]
titles = ['Decision Tree', 'KNN', 'Logistic Regression']

for ax, cm, title in zip(axes, matrices, titles):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, cbar=False)
    ax.set_title(title, fontweight='bold')
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')

plt.tight_layout()
plt.show()

print("✅ Confusion matrices visualized!")

In [ ]:
# Classification reports
print("📋 CLASSIFICATION REPORTS:\n")

print("="*60)
print("DECISION TREE")
print("="*60)
print(classification_report(y_test, y_pred_dt, target_names=target_names))

print("="*60)
print("K-NEAREST NEIGHBORS")
print("="*60)
print(classification_report(y_test, y_pred_knn, target_names=target_names))

print("="*60)
print("LOGISTIC REGRESSION")
print("="*60)
print(classification_report(y_test, y_pred_lr, target_names=target_names))

## Phase 7: Performance Metrics Comparison

Compare all models using multiple evaluation metrics.

In [ ]:
# Calculate all metrics
print("📊 PERFORMANCE METRICS COMPARISON:\n")

metrics_data = []

# Decision Tree
metrics_data.append({
    'Model': 'Decision Tree',
    'Accuracy': accuracy_score(y_test, y_pred_dt),
    'Precision': precision_score(y_test, y_pred_dt, average='weighted'),
    'Recall': recall_score(y_test, y_pred_dt, average='weighted'),
    'F1-Score': f1_score(y_test, y_pred_dt, average='weighted')
})

# KNN
metrics_data.append({
    'Model': 'KNN (k=5)',
    'Accuracy': accuracy_score(y_test, y_pred_knn),
    'Precision': precision_score(y_test, y_pred_knn, average='weighted'),
    'Recall': recall_score(y_test, y_pred_knn, average='weighted'),
    'F1-Score': f1_score(y_test, y_pred_knn, average='weighted')
})

# Logistic Regression
metrics_data.append({
    'Model': 'Logistic Regression',
    'Accuracy': accuracy_score(y_test, y_pred_lr),
    'Precision': precision_score(y_test, y_pred_lr, average='weighted'),
    'Recall': recall_score(y_test, y_pred_lr, average='weighted'),
    'F1-Score': f1_score(y_test, y_pred_lr, average='weighted')
})

metrics_df = pd.DataFrame(metrics_data)
print(metrics_df.to_string(index=False))

In [ ]:
# Visualize metrics comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(metrics_df))
width = 0.2

ax.bar(x - 1.5*width, metrics_df['Accuracy'], width, label='Accuracy')
ax.bar(x - 0.5*width, metrics_df['Precision'], width, label='Precision')
ax.bar(x + 0.5*width, metrics_df['Recall'], width, label='Recall')
ax.bar(x + 1.5*width, metrics_df['F1-Score'], width, label='F1-Score')

ax.set_xlabel('Models', fontweight='bold')
ax.set_ylabel('Score', fontweight='bold')
ax.set_title('Classification Models - Performance Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics_df['Model'])
ax.legend()
ax.set_ylim([0.8, 1.05])
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✅ Metrics comparison visualized!")

## Phase 8: Practical Tests

Complete all 5 tests to verify your learning outcomes.

In [ ]:
# TEST 1: TRAIN-TEST SPLIT
print("🧪 TEST 1: TRAIN-TEST SPLIT")
try:
    assert X_train.shape[0] > 0, "Training set is empty!"
    assert X_test.shape[0] > 0, "Test set is empty!"
    assert X_train.shape[1] == X_test.shape[1], "Feature mismatch!"
    assert len(y_train) == X_train.shape[0], "Target mismatch!"
    print(f"✅ TEST 1 PASSED: Train size {X_train.shape[0]}, Test size {X_test.shape[0]}")
    test1_result = "PASSED"
except AssertionError as e:
    print(f"❌ TEST 1 FAILED: {e}")
    test1_result = "FAILED"

In [ ]:
# TEST 2: DECISION TREE
print("\n🧪 TEST 2: DECISION TREE CLASSIFIER")
try:
    assert dt_accuracy > 0.7, "Decision Tree accuracy too low!"
    assert len(y_pred_dt) == len(y_test), "Prediction size mismatch!"
    assert len(dt_model.feature_importances_) == 4, "Feature importance size mismatch!"
    print(f"✅ TEST 2 PASSED: Accuracy {dt_accuracy:.4f}")
    test2_result = "PASSED"
except AssertionError as e:
    print(f"❌ TEST 2 FAILED: {e}")
    test2_result = "FAILED"

In [ ]:
# TEST 3: KNN CLASSIFIER
print("\n🧪 TEST 3: KNN CLASSIFIER")
try:
    assert knn_accuracy > 0.7, "KNN accuracy too low!"
    assert len(y_pred_knn) == len(y_test), "Prediction size mismatch!"
    assert knn_model.n_neighbors == 5, "K value incorrect!"
    print(f"✅ TEST 3 PASSED: Accuracy {knn_accuracy:.4f}")
    test3_result = "PASSED"
except AssertionError as e:
    print(f"❌ TEST 3 FAILED: {e}")
    test3_result = "FAILED"

In [ ]:
# TEST 4: LOGISTIC REGRESSION
print("\n🧪 TEST 4: LOGISTIC REGRESSION")
try:
    assert lr_accuracy > 0.7, "Logistic Regression accuracy too low!"
    assert len(y_pred_lr) == len(y_test), "Prediction size mismatch!"
    assert lr_model.coef_.shape[0] == 3, "Coefficient shape incorrect!"
    print(f"✅ TEST 4 PASSED: Accuracy {lr_accuracy:.4f}")
    test4_result = "PASSED"
except AssertionError as e:
    print(f"❌ TEST 4 FAILED: {e}")
    test4_result = "FAILED"

In [ ]:
# TEST 5: METRICS COMPARISON
print("\n🧪 TEST 5: METRICS COMPARISON")
try:
    assert len(metrics_df) == 3, "Should have 3 models!"
    assert all(metrics_df['Accuracy'] > 0.7), "All accuracies should be > 0.7!"
    assert all((metrics_df['Precision'] >= 0) & (metrics_df['Precision'] <= 1)), "Invalid precision!"
    assert all((metrics_df['Recall'] >= 0) & (metrics_df['Recall'] <= 1)), "Invalid recall!"
    assert all((metrics_df['F1-Score'] >= 0) & (metrics_df['F1-Score'] <= 1)), "Invalid F1-Score!"
    print(f"✅ TEST 5 PASSED: All metrics valid and calculated")
    test5_result = "PASSED"
except AssertionError as e:
    print(f"❌ TEST 5 FAILED: {e}")
    test5_result = "FAILED"

## Results Summary

### Test Results

In [ ]:
# Create summary report
test_summary = pd.DataFrame([
    {'Test': 'Test 1: Train-Test Split', 'Result': test1_result},
    {'Test': 'Test 2: Decision Tree', 'Result': test2_result},
    {'Test': 'Test 3: KNN', 'Result': test3_result},
    {'Test': 'Test 4: Logistic Regression', 'Result': test4_result},
    {'Test': 'Test 5: Metrics Comparison', 'Result': test5_result}
])

print("\n" + "="*60)
print("TEST RESULTS SUMMARY")
print("="*60)
print(test_summary.to_string(index=False))

passed = sum([1 for r in [test1_result, test2_result, test3_result, test4_result, test5_result] if r == 'PASSED'])
total = 5
print(f"\n📊 SCORE: {passed}/{total} TESTS PASSED ({passed*100/total:.0f}%)")
print("="*60)

## Reflections & Learnings

### Key Insights

1. **Decision Trees:**
   - Interpretable but prone to overfitting
   - No feature scaling needed
   - Good for identifying important features

2. **K-Nearest Neighbors:**
   - Simple but computationally expensive
   - Requires feature scaling
   - K value significantly affects performance

3. **Logistic Regression:**
   - Fast and interpretable
   - Assumes linear decision boundary
   - Provides probability estimates

4. **Model Evaluation:**
   - No single metric tells the whole story
   - Confusion matrix reveals specific error types
   - Precision vs Recall trade-off is important

### Best Practices

- Always split data into train/test sets
- Scale features for distance-based algorithms
- Use stratified split for imbalanced datasets
- Compare multiple models before choosing one
- Consider business context when selecting metrics

## Submission Checklist

Before submitting your practical, verify:

### Code Completion
- [ ] Phase 1 (Data Exploration): Completed
- [ ] Phase 2 (Train-Test Split): Completed with scaling
- [ ] Phase 3 (Decision Tree): Completed with hyperparameters
- [ ] Phase 4 (KNN): Completed with K optimization
- [ ] Phase 5 (Logistic Regression): Completed with coefficient analysis
- [ ] Phase 6 (Evaluation): Confusion matrices and reports
- [ ] Phase 7 (Metrics Comparison): All metrics calculated

### Test Results
- [ ] All 5 tests passed ✅
- [ ] Test output clearly visible
- [ ] Error handling demonstrated
- [ ] Performance metrics recorded

### Documentation
- [ ] Student information filled
- [ ] Learning outcomes checklist marked
- [ ] Reflections written
- [ ] Code is well-commented
- [ ] All output is visible and clear

### Files
- [ ] Notebook saved as `Practical_4_Complete_Notebook.ipynb`
- [ ] PDF exported from notebook
- [ ] No errors in notebook execution

---

**Practical Status:** Ready for submission ✅